In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model_name="llama-3.1-8b-instant",
    temperature=0,
    groq_api_key='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'
)

In [2]:
%pip install -qU langchain-community beautifulsoup4

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\gaikw\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

page_url = "https://python.langchain.com/docs/how_to/chatbots_memory/"

loader = WebBaseLoader(web_paths=[page_url])
docs = []
async for doc in loader.alazy_load():
    docs.append(doc)

assert len(docs) == 1
doc = docs[0]

USER_AGENT environment variable not set, consider setting it to identify your requests.
Fetching pages: 100%|##########| 1/1 [00:00<00:00,  2.42it/s]


In [4]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.walmart.com/us/jobs/WD2193135-staff-data-scientist")
page_data = loader.load().pop().page_content
print(page_data)

Staff, Data Scientist | Walmart Careers             















  

    Careers    All Career Areas  Back  Stores & Clubs  Sam’s Club Jobs  Sam’s Club Management Jobs  Walmart Management Jobs  Walmart Store Jobs   Distribution, Fulfillment, & Drivers  Distribution and Fulfillment Centers  Drivers & Transportation  Fleet Maintenance  Flex Associate  Manufacturing   Healthcare  Corporate Healthcare Roles  Optical  Pharmacy   Technology  Cybersecurity  Data Science and Analytics  Information Technology  Project and Program Management - Technology  Software Development and Engineering  UX Design  View all Technology   Corporate  Accounting and Finance  Administrative and Support Services  Asset Protection  Aviation and Travel  Business Operations  Corporate Affairs and Communications  Customer Service and Call Center  Data Analytics and Business Intelligence  Engineering  Ethics and Compliance  Global Investigations and Security  Human Resources  Installation, Maintenance and Utilities

In [5]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [6]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'Staff, Data Scientist',
  'experience': {'required': ["4 years' experience in an analytics related field",
    "2 years' experience in an analytics related field",
    "6 years' experience in an analytics or related field"],
   'optional': ['Data science, machine learning, optimization models, PhD in Machine Learning, Computer Science, Information Technology, Operations Research, Statistics, Applied Mathematics, Econometrics, Successful completion of one or more assessments in Python, Spark, Scala, or R, Using open source frameworks (for example, scikit learn, tensorflow, torch)']},
  'skills': {'required': ['Python', 'SQL', 'GCP', 'PowerBI'],
   'optional': ['Data science, machine learning, optimization models, PhD in Machine Learning, Computer Science, Information Technology, Operations Research, Statistics, Applied Mathematics, Econometrics, Successful completion of one or more assessments in Python, Spark, Scala, or R, Using open source frameworks (for example, scikit le

In [7]:
type(json_res)

list

In [8]:

json_parser = JsonOutputParser()

try:
    json_res = json_parser.parse(res.content)

    # Flatten result if returned as a single-item list
    if isinstance(json_res, list) and len(json_res) == 1:
        json_res = json_res[0]

    print("✅ Extracted job posting:")
    print(json_res)

except Exception as e:
    print("❌ Failed to parse LLM response:", e)
    print("Raw output:")
    print(res.content)


✅ Extracted job posting:
{'role': 'Staff, Data Scientist', 'experience': {'required': ["4 years' experience in an analytics related field", "2 years' experience in an analytics related field", "6 years' experience in an analytics or related field"], 'optional': ['Data science, machine learning, optimization models, PhD in Machine Learning, Computer Science, Information Technology, Operations Research, Statistics, Applied Mathematics, Econometrics, Successful completion of one or more assessments in Python, Spark, Scala, or R, Using open source frameworks (for example, scikit learn, tensorflow, torch)']}, 'skills': {'required': ['Python', 'SQL', 'GCP', 'PowerBI'], 'optional': ['Data science, machine learning, optimization models, PhD in Machine Learning, Computer Science, Information Technology, Operations Research, Statistics, Applied Mathematics, Econometrics, Successful completion of one or more assessments in Python, Spark, Scala, or R, Using open source frameworks (for example, sci

In [9]:
type(json_res)

dict

In [10]:
%pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\gaikw\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [11]:
import pandas as pd

df = pd.read_csv("My_portfolio.csv")
df

,Techstack,Links
0,"Python, Pandas, NumPy, sqlite3, json",https://github.com/vaibhavgaikwad7/Impact-of-w...
1,"R, Machine Learning",https://github.com/vaibhavgaikwad7/Energy-Cons...
2,"Python, Machine Learning, Scikit-learn, Tensor...",https://github.com/vaibhavgaikwad7/Credibility...
3,"Tableau, SQL",https://github.com/vaibhavgaikwad7/AI-Job-Mark...
4,"Python, Deep Learning, LSTM, Neural Netork",https://github.com/vaibhavgaikwad7/Portfolio-O...


In [12]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [13]:
links = collection.query(query_texts=['Experience in Python', 'Experience in Machine Learning'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://github.com/vaibhavgaikwad7/Credibility-Detection-of-Health-Web-Blogs-Using-Explainable-AI'},
  {'links': 'https://github.com/vaibhavgaikwad7/Portfolio-Optimization-Using-Bidirectional-CNN-LSTM-with-Attention-Mechanism'}],
 [{'links': 'https://github.com/vaibhavgaikwad7/Energy-Consumption-Predictor'},
  {'links': 'https://github.com/vaibhavgaikwad7/Credibility-Detection-of-Health-Web-Blogs-Using-Explainable-AI'}]]

In [14]:
job = json_res
job['skills']

{'required': ['Python', 'SQL', 'GCP', 'PowerBI'],
 'optional': ['Data science, machine learning, optimization models, PhD in Machine Learning, Computer Science, Information Technology, Operations Research, Statistics, Applied Mathematics, Econometrics, Successful completion of one or more assessments in Python, Spark, Scala, or R, Using open source frameworks (for example, scikit learn, tensorflow, torch)']}

In [15]:
# Flatten the job skill requirements into a list of query strings
skill_queries = job['skills']['required'] + job['skills']['optional']

# Query the vector database using those skill phrases
links = collection.query(
    query_texts=skill_queries,
    n_results=2
).get('metadatas', [])

# Print matched portfolio links
print(links)


[[{'links': 'https://github.com/vaibhavgaikwad7/Credibility-Detection-of-Health-Web-Blogs-Using-Explainable-AI'}, {'links': 'https://github.com/vaibhavgaikwad7/Portfolio-Optimization-Using-Bidirectional-CNN-LSTM-with-Attention-Mechanism'}], [{'links': 'https://github.com/vaibhavgaikwad7/AI-Job-Market-Trends-in-USA-using-Tableau'}, {'links': 'https://github.com/vaibhavgaikwad7/Energy-Consumption-Predictor'}], [{'links': 'https://github.com/vaibhavgaikwad7/Energy-Consumption-Predictor'}, {'links': 'https://github.com/vaibhavgaikwad7/Portfolio-Optimization-Using-Bidirectional-CNN-LSTM-with-Attention-Mechanism'}], [{'links': 'https://github.com/vaibhavgaikwad7/Energy-Consumption-Predictor'}, {'links': 'https://github.com/vaibhavgaikwad7/AI-Job-Market-Trends-in-USA-using-Tableau'}], [{'links': 'https://github.com/vaibhavgaikwad7/Credibility-Detection-of-Health-Web-Blogs-Using-Explainable-AI'}, {'links': 'https://github.com/vaibhavgaikwad7/Portfolio-Optimization-Using-Bidirectional-CNN-LSTM-

In [16]:
# Flatten and deduplicate the list of result links
flat_links = [item['links'] for sublist in links for item in sublist]
unique_links = list(set(flat_links))

# Pretty print
print("Matched portfolio links:")
for link in unique_links:
    print("-", link)


Matched portfolio links:
- https://github.com/vaibhavgaikwad7/Energy-Consumption-Predictor
- https://github.com/vaibhavgaikwad7/Credibility-Detection-of-Health-Web-Blogs-Using-Explainable-AI
- https://github.com/vaibhavgaikwad7/Portfolio-Optimization-Using-Bidirectional-CNN-LSTM-with-Attention-Mechanism
- https://github.com/vaibhavgaikwad7/AI-Job-Market-Trends-in-USA-using-Tableau


In [17]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Vaibhav, a recently graduated student from Syracuse University - with the degree of Masters in Applied Data Science. 
        Your job is to write a cold email to the HR of that job description telling that you are fit for this role.
        Also add the most relevant ones from the following links to showcase Your's portfolio: {unique_list}
        Remember you are Vaibhav, A data enthusiast looking for full time role, Your former experience include 1) AI researcher at HeadOn (description - Developed and fine-tuned LLMs (BERT, GPT) to generate meeting summaries and classify discussion sentiment (positive/negative). Integrated real-time inference into the product using REST APIs, reducing manual note-taking by 80%.Achieved 90%+ accuracy in sentiment detection through iterative model optimization and feedback loops.). 
        2) Data Science Researcher at NEXIS Student Labs (description - Built robust ETL pipelines using Python and SQL to process large volumes of medical diagnostic data. Developed predictive models that improved diagnostic accuracy and early detection rates by 25%. Designed interactive Power BI dashboards to visualize patient trends and clinical insights.)
        3) Teaching Assistant at Syracuse University (description - •	Supported 90+ students in machine learning and forecasting courses using Python and Power BI.). 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "unique_list": links})
print(res.content)

Subject: Application for Staff Data Scientist Role at Sam's

Dear Hiring Manager,

I am writing to express my interest in the Staff Data Scientist role at Sam's, as advertised on [Job Board]. With a strong background in data science and a passion for driving business growth through data-driven insights, I am confident that I would be an excellent fit for this position.

As a recent graduate of Syracuse University's Master's program in Applied Data Science, I have developed a solid foundation in data analysis, machine learning, and data visualization. My experience as an AI researcher at HeadOn, where I developed and fine-tuned LLMs to generate meeting summaries and classify discussion sentiment, has given me a deep understanding of natural language processing and its applications in real-world scenarios.

In my previous role as a Data Science Researcher at NEXIS Student Labs, I built robust ETL pipelines using Python and SQL to process large volumes of medical diagnostic data. I also d